In [2]:
!pip install mysql-connector-python

In [80]:
import mysql.connector

In [4]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [9]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import mysql.connector

# Database connection details
host = "127.0.0.1"         # Replace with your host, e.g., 'localhost'
user = "root"              # Replace with your MySQL username
password = "YOUR PASSWORD"     # Replace with your MySQL password
database = "schoolresults" # Replace with your database name

# Load the CSV files
student_df = pd.read_csv("C:/Users/USER/Documents/dataset/student information/student.csv")
student_profile_df = pd.read_csv("C:/Users/USER/Documents/dataset/student information/student_profile.csv")
courses_df = pd.read_csv("courses.csv")
assessment_df = pd.read_csv("C:/Users/USER/Documents/dataset/assessment/Consolidated Assessment.csv")

print("CSV files loaded successfully.")

# Connect to the database
conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
cursor = conn.cursor()

try:
    # Clear staging tables before inserting new data
    staging_tables = ['Student_Staging', 'Student_Profile_Staging', 'Courses_Staging', 'Assessment_Staging']
    for table in staging_tables:
        cursor.execute(f"DELETE FROM {table}")
    conn.commit()
    print("Staging tables cleared.")

    # Insert data into Student_Staging
    for _, row in student_df.iterrows():
        insert_query = """
        INSERT INTO Student_Staging (admission_no, first_name, last_name)
        VALUES (%s, %s, %s)
        """
        cursor.execute(insert_query, (row['admission_no'], row['first_name'], row['last_name']))
    conn.commit()
    print("Data loaded into Student_Staging successfully.")

    # Insert data into Student_Profile_Staging
    for _, row in student_profile_df.iterrows():
        insert_query = """
        INSERT INTO Student_Profile_Staging (student, age, gender, classroom, sit_number)
        VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (row['student'], row['age'], row['gender'], row['classroom'], row['sit_number']))
    conn.commit()
    print("Data loaded into Student_Profile_Staging successfully.")

    # Insert data into Courses_Staging
    for _, row in courses_df.iterrows():
        insert_query = """
        INSERT INTO Courses_Staging (course_code, course_name)
        VALUES (%s, %s)
        """
        cursor.execute(insert_query, (row['course_code'], row['course name']))
    conn.commit()
    print("Data loaded into Courses_Staging successfully.")

    # Insert data into Assessment_Staging
    for _, row in assessment_df.iterrows():
        insert_query = """
        INSERT INTO Assessment_Staging (course_code, student, first_ca, second_ca, exam, total)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (row['course_code'], row['student'], row['first_ca'], row['second_ca'], row['exam'], row['total']))
    conn.commit()
    print("Data loaded into Assessment_Staging successfully.")

    # Synchronize staging tables with main tables
    print("Starting synchronization with main tables...")

    # Synchronize Student table
    cursor.execute("""
        DELETE FROM Student
        WHERE admission_no NOT IN (SELECT admission_no FROM Student_Staging);
    """)
    cursor.execute("""
        INSERT INTO Student (admission_no, first_name, last_name)
        SELECT admission_no, first_name, last_name
        FROM Student_Staging
        WHERE admission_no NOT IN (SELECT admission_no FROM Student);
    """)
    conn.commit()
    print("Student table synchronized.")

    # Synchronize Student_Profile table
    cursor.execute("""
        DELETE FROM Student_Profile
        WHERE student NOT IN (SELECT student FROM Student_Profile_Staging);
    """)
    cursor.execute("""
        INSERT INTO Student_Profile (student, age, gender, classroom, sit_number)
        SELECT student, age, gender, classroom, sit_number
        FROM Student_Profile_Staging
        WHERE student NOT IN (SELECT student FROM Student_Profile);
    """)
    conn.commit()
    print("Student_Profile table synchronized.")

    # Synchronize Courses table
    cursor.execute("""
        DELETE FROM Courses
        WHERE course_code NOT IN (SELECT course_code FROM Courses_Staging);
    """)
    cursor.execute("""
        INSERT INTO Courses (course_code, course_name)
        SELECT course_code, course_name
        FROM Courses_Staging
        WHERE course_code NOT IN (SELECT course_code FROM Courses);
    """)
    conn.commit()
    print("Courses table synchronized.")

    # Synchronize Assessment table
    cursor.execute("""
        DELETE FROM Assessment
        WHERE course_code NOT IN (SELECT course_code FROM Assessment_Staging)
        OR student NOT IN (SELECT student FROM Assessment_Staging);
    """)
    cursor.execute("""
        INSERT INTO Assessment (course_code, student, first_ca, second_ca, exam, total)
        SELECT course_code, student, first_ca, second_ca, exam, total
        FROM Assessment_Staging
        WHERE (course_code, student) NOT IN (
            SELECT course_code, student FROM Assessment
        );
    """)
    conn.commit()
    print("Assessment table synchronized.")

    # End of synchronization
    print("Synchronization with main tables completed.")

except mysql.connector.Error as error:
    print(f"Error: {error}")

finally:
    # Close the database connection
    cursor.close()
    conn.close()
    print("Database connection closed. Staging and synchronization completed successfully.")

CSV files loaded successfully.
Staging tables cleared.
Data loaded into Student_Staging successfully.
Data loaded into Student_Profile_Staging successfully.
Data loaded into Courses_Staging successfully.
Data loaded into Assessment_Staging successfully.
Starting synchronization with main tables...
Student table synchronized.
Student_Profile table synchronized.
Courses table synchronized.
Assessment table synchronized.
Synchronization with main tables completed.
Database connection closed. Staging and synchronization completed successfully.
